### Load Detections from Alerce ZTF2 Database

In [1]:
import pandas as pd
import numpy as np
from astropy.units import deg
import os
from datetime import date
import time
from collections import Counter
from tqdm.auto import tqdm

# Libraries for getting Alerce data out of ZTF2 database
import json
import psycopg2
from alerce.api import AlerceAPI

# MSE imports
from utils import range_inc
from astro_utils import date_to_mjd, mjd_to_date
from ztf_data import load_ztf_det, load_ztf_det_year, ztf_det_add_dir
from ra_dec import radec2dir, radec_diff, direction_diff

In [2]:
# Load 2018 and 2019 data
df_2018 = load_ztf_det_year(2018)
df_2019 = load_ztf_det_year(2019)
df_2020 = load_ztf_det_year(2020)

Loaded ../data/ztf/ztf-detections-2018.h5 from disk.
Loaded ../data/ztf/ztf-detections-2019.h5 from disk.
Loaded ../data/ztf/ztf-detections-2020.h5 from disk.


In [3]:
# Combine frames into one DataFrame
df = pd.concat([df_2018, df_2019, df_2020])
df

,objectid,candidateid,mjd,ra,dec,asteroid_prob
0,ZTF18aahbsho,518255634615010004,58272.255637,247.480690,34.027481,0.998269
1,ZTF18aaxpmnp,517454781815010004,58271.454780,308.738474,32.164302,0.935730
2,ZTF18aaxjdey,517457175915015004,58271.457176,299.974765,28.940493,0.915364
3,ZTF18aawceqv,516377973215015007,58270.377974,302.335815,34.731209,0.686181
4,ZTF18aawceqv,516434764515015021,58270.434768,302.335813,34.731214,0.686181
...,...,...,...,...,...,...
17265,ZTF20aarbpjv,1151110874715015000,58905.110880,30.868254,5.255209,1.000000
17266,ZTF20aarbple,1151111784015015003,58905.111782,29.698298,20.444825,1.000000
17267,ZTF20aarbpli,1151111783215015001,58905.111782,33.721698,20.553823,1.000000
17268,ZTF20aarbpmt,1151111784315015004,58905.111782,29.737316,19.527889,1.000000


In [ ]:
# Extract mjd, ra, and dec as vectors of astropy angles
mjd_ztf = df.mjd.values
ra_ztf = df.ra.values * deg
dec_ztf = df.dec.values * deg

In [ ]:
# Summarize number data availability by days
mjds_distinct = sorted(set(np.int32(mjd_ztf)))
dates_distinct = [mjd_to_date(int(mjd)) for mjd in mjds_distinct]

# Count number of days in each calendar month
months = Counter()
for dt in dates_distinct:
    months[(dt.year, dt.month)] += 1
months

In [ ]:
# Add the direction to the combined Alerce DataFrame using radec2dir()
u_ztf = radec2dir(ra=ra_ztf, dec=dec_ztf, obstime_mjd=mjd_ztf)

In [ ]:
u_ztf.shape

In [ ]:
# Add these directions to the DataFrame
df.insert(loc=5, column='ux', value=0.0)
df.insert(loc=6, column='uy', value=0.0)
df.insert(loc=7, column='uz', value=0.0)
df

In [ ]:
# Save computed directions to DataFrame
df.ux = u_ztf[0]
df.uy = u_ztf[1]
df.uz = u_ztf[2]

In [ ]:
file_name='../data/ztf/ztf-detections.h5'
df.to_hdf(file_name, key='df', mode='w')

In [ ]:
df.columns

In [ ]:
df.columns.find

In [ ]:
df.columns.get_loc('dec')

In [4]:
ztf_det_add_dir(df=df, file_name='ztf-detections.h5', dir_name='../data/ztf')

/home/michael/anaconda3/envs/kepler/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['objectid', 'candidateid'], dtype='object')]

  encoding=encoding,
